# How RMG generates reaction templates?

Han, Kehang (hkh12@mit.edu)

Nov.16, 2015

## Motivation
During RMG new feature development, it is occasionally observed that new change made same kinetic model produce different simulation results, e.g., selecting different edge species as the new core species. The cause usually is although RMG gives same reactions but generates different kinetics. 

For instance, the newly-developed branch `edge_inchi_rxn` produces almost same models as `master` branch does except some divergence which is found to be caused by different kinects for same reactions estimated by RMG. Further investigation shows the different kinetics stemmed from different reaction templates matched. Therefore, a full understanding of how RMG generates reation templates will help solve the discrepancy between the models from `edhe_inchi_rxn` and those from `master`.

## Roadmap

This post will go through the entire process of generating reaction templates in RMG, which is consist of two main steps as shown below.

- Set-up

- Make reactions

## Set-up
Before investigating, several pre-steps should be needed for set-up.

- import all the necessary modules

- create RMG object and load database needed

- create two reactants (C=CC=C and C=CCC go through disproportionation) for reacting research

In [15]:
%matplotlib inline
from rmgpy.rmg.main import RMG, CoreEdgeReactionModel
from rmgpy.data.rmg import RMGDatabase, database
from rmgpy.data.base import ForbiddenStructureException
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from IPython.display import display
import os

In [16]:
# set-up RMG object
rmg = RMG()
rmg.reactionModel = CoreEdgeReactionModel()

# load kinetic database and forbidden structures
rmg.database = RMGDatabase()
home_path = os.getenv('HOME')
path = os.path.join(home_path, 'Code', 'rmgmaster', 'RMG-database', 'input')

# forbidden structure loading
rmg.database.loadForbiddenStructures(os.path.join(path, 'forbiddenStructures.py'))
# kinetics family Disproportionation loading
rmg.database.loadKinetics(os.path.join(path, 'kinetics'), \
                          kineticsFamilies=['Disproportionation'])

In [17]:
# create reactants (molecules): C=CC=C and C=CCC
molA = Molecule().fromSMILES("C=CC=C")
molB = Molecule().fromSMILES("C=CCC")
reactants = [molA, molB]
print "molA:"
display(molA)
print "molB:"
display(molB)

molA:


molB:


## Make reactions
The reaction-making step into five sub-steps,

- Pick reaction family

- Map reactants to family_template reactants

- Create product strudtures and reactions based on mapping

- Remove duplicate reactions

- Get pairs and template for each remaining reaction

As you will see, the final kinetics template is actually determined in the second sub-step: the first one of all mappings that lead to same reactions will win because of duplicates removal. 

### Pick reaction family

In [18]:
# pick out the reacting family
families = rmg.database.kinetics.families
disprop_family = families['Disproportionation']

### Map reactants to family_template reactants

In [19]:
# map reactants to the reacting family's template reactants
family_template = disprop_family.reverseTemplate
mappingsA = disprop_family.\
_KineticsFamily__matchReactantToTemplate(molA, family_template.reactants[1])
mappingsB = disprop_family.\
_KineticsFamily__matchReactantToTemplate(molB, family_template.reactants[0])

After using the 1st mapping for molA, `*2` is assigned to the middle carbon, which is **from** the carbon losing one `H` atom. This mapping leads molA (`C=CC=C`) to pair with `C[CH]C=C`.

In [20]:
# assign the labels to atoms based on first mapping
mapA = mappingsA[0]
molA.clearLabeledAtoms()
for atom, templateAtom in mapA.iteritems():
    atom.label = templateAtom.label

print "molA with first mapping: \n" + str(molA.toAdjacencyList())

molA with first mapping: 
1  *2 C u0 p0 c0 {2,S} {3,D} {5,S}
2     C u0 p0 c0 {1,S} {4,D} {6,S}
3  *3 C u0 p0 c0 {1,D} {7,S} {8,S}
4     C u0 p0 c0 {2,D} {9,S} {10,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {2,S}
7     H u0 p0 c0 {3,S}
8     H u0 p0 c0 {3,S}
9     H u0 p0 c0 {4,S}
10    H u0 p0 c0 {4,S}



After using the 3rd mapping for molA, `*2` is assigned to the terminal carbon, which is **from** the carbon losing one `H` atom. This mapping leads molA (`C=CC=C`) to pair with `[CH2]CC=C`.

In [21]:
# assign the labels to atoms based on third mapping
mapA = mappingsA[2]
molA.clearLabeledAtoms()
for atom, templateAtom in mapA.iteritems():
    atom.label = templateAtom.label

print "molA with third mapping: \n" + str(molA.toAdjacencyList())

molA with third mapping: 
1  *3 C u0 p0 c0 {2,S} {3,D} {5,S}
2     C u0 p0 c0 {1,S} {4,D} {6,S}
3  *2 C u0 p0 c0 {1,D} {7,S} {8,S}
4     C u0 p0 c0 {2,D} {9,S} {10,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {2,S}
7     H u0 p0 c0 {3,S}
8     H u0 p0 c0 {3,S}
9     H u0 p0 c0 {4,S}
10    H u0 p0 c0 {4,S}



Similar to molB, we have:

In [22]:
# assign the labels to atoms based on first mapping
mapB = mappingsB[0]
molB.clearLabeledAtoms()
for atom, templateAtom in mapB.iteritems():
    atom.label = templateAtom.label
print "molB with first mapping: \n" + str(molB.toAdjacencyList())
# assign the labels to atoms based on third mapping
mapB = mappingsB[2]
molB.clearLabeledAtoms()
for atom, templateAtom in mapB.iteritems():
    atom.label = templateAtom.label
print "molB with third mapping: \n" + str(molB.toAdjacencyList())

molB with first mapping: 
1  *1 C u0 p0 c0 {2,S} {3,S} {5,S} {6,S}
2     C u0 p0 c0 {1,S} {7,S} {8,S} {9,S}
3     C u0 p0 c0 {1,S} {4,D} {10,S}
4     C u0 p0 c0 {3,D} {11,S} {12,S}
5  *4 H u0 p0 c0 {1,S}
6     H u0 p0 c0 {1,S}
7     H u0 p0 c0 {2,S}
8     H u0 p0 c0 {2,S}
9     H u0 p0 c0 {2,S}
10    H u0 p0 c0 {3,S}
11    H u0 p0 c0 {4,S}
12    H u0 p0 c0 {4,S}

molB with third mapping: 
1     C u0 p0 c0 {2,S} {3,S} {5,S} {6,S}
2  *1 C u0 p0 c0 {1,S} {7,S} {8,S} {9,S}
3     C u0 p0 c0 {1,S} {4,D} {10,S}
4     C u0 p0 c0 {3,D} {11,S} {12,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {1,S}
7  *4 H u0 p0 c0 {2,S}
8     H u0 p0 c0 {2,S}
9     H u0 p0 c0 {2,S}
10    H u0 p0 c0 {3,S}
11    H u0 p0 c0 {4,S}
12    H u0 p0 c0 {4,S}



### Create product structures and reactions based on mapping

Usually different mapping combinations `(mapA, mapB)` will lead to different products, i.e., different reactions, so which mapping to be applied first doesn't really matter. However, the reaction below is special enough that different mappings lead to same reaction. In the other words, the very same reaction will have different kinetics templates matched because of different mappings. If reaction duplicates removal is only based on isomorphic checking of reactants and products, which is actually the case, the final kinetics template will be determined by the very first mapping, which may result in non-deterministic behavior if mappings' order varies somehow.

In [23]:
# create product structures by applying some mapping combination `(mapA, mapB)`
print "mapping combination: 1st mapA and 1st mapB"
for mapA in mappingsA[:1]:
    for mapB in mappingsB[:1]:
        reactantStructures = [molA, molB]
        try:
            productStructures = \
            disprop_family._KineticsFamily__generateProductStructures(reactantStructures, \
                                                                      [mapA, mapB], \
                                                                      False)
        except ForbiddenStructureException:
            pass
        else:
            if productStructures is not None:
                rxn1 = disprop_family._KineticsFamily__createReaction(reactantStructures, \
                                                                     productStructures, \
                                                                     False)
                if rxn1: print rxn1

mapping combination: 1st mapA and 1st mapB
<Molecule "C=C[CH]C"> + <Molecule "[CH2]CC=C"> <=> <Molecule "C=CC=C"> + <Molecule "C=CCC">


In [24]:
# create product structures by applying another mapping combination `(mapA, mapB)`
print "mapping combination: 3rd mapA and 3rd mapB"
for mapA in mappingsA[2:3]:
    for mapB in mappingsB[2:3]:
        reactantStructures = [molA, molB]
        try:
            productStructures = \
            disprop_family.\
            _KineticsFamily__generateProductStructures(reactantStructures, \
                                                       [mapA, mapB], \
                                                       False)
        except ForbiddenStructureException:
            pass
        else:
            if productStructures is not None:
                rxn3 = disprop_family.\
                _KineticsFamily__createReaction(reactantStructures, \
                                                productStructures, False)
                if rxn3: print rxn3

mapping combination: 3rd mapA and 3rd mapB
<Molecule "[CH2]CC=C"> + <Molecule "C=C[CH]C"> <=> <Molecule "C=CC=C"> + <Molecule "C=CCC">


### Remove duplicates
This step is to only keep the first unique reaction and eliminate other duplicates in the sense of products isomorphism.

We've already seen, two different mapping combinations `(1st mapA & 1st mapB) vs.(3rd mapA & 3rd mapB)` lead to same product structures, thus same reactions. However if looking closer, the pairs of two reactions are actually different. These two mapping combinations stand for different reactant-product pair thus different transition states, probably having very different kinetics. However, current RMG `master` branch chooses to keep only the first occurance which is `(1st mapA & 1st mapB)` instead of `(3rd mapA & 3rd mapB)`.

### Get pairs and template for remaining reactions
In this example, since `rxn1` is generated firstly, `rxn1` is only kept for reaction `[CH2]CC=C + C=C[CH]C <=> C=CC=C + C=CCC`. So the template would be using `rxn1's` template as well the kinetics determined by that in further steps.

In [25]:
print disprop_family.getReactionPairs(rxn1)
print disprop_family.getReactionTemplate(rxn1)

[[Molecule(SMILES="C=C[CH]C"), Molecule(SMILES="C=CCC")], [Molecule(SMILES="[CH2]CC=C"), Molecule(SMILES="C=CC=C")]]
[<Entry index=53 label="C_rad/H/OneDeC">, <Entry index=99 label="C/H2/De_Csrad">]


But if somehow, `rxn3` is generated firstly, the final kinetics for `[CH2]CC=C + C=C[CH]C <=> C=CC=C + C=CCC` will be much different because of using a different template.

In [26]:
print disprop_family.getReactionPairs(rxn3)
print disprop_family.getReactionTemplate(rxn3)

[[Molecule(SMILES="[CH2]CC=C"), Molecule(SMILES="C=CCC")], [Molecule(SMILES="C=C[CH]C"), Molecule(SMILES="C=CC=C")]]
[<Entry index=38 label="C_rad/H2/Cs">, <Entry index=287 label="Cmethyl_Csrad/H/Cd">]


## Comments
The current method of determining the template is risky in some situations below:

- where rxn1 is much slower or much faster than rxn3, `[CH2]CC=C + C=C[CH]C <=> C=CC=C + C=CCC` cannot really be represented by rxn1 because `degeneracy = 2` (1 for rxn1, 1 for rxn3) is not appropriate any more.

Possible solutions:

- store all the templates (from rxn1 and from rxn3) and kinetics should be added not simply multiplied by degeneracy

- your inputs:...